In [4]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

from typing_extensions import override

# API 키 정보 로드
load_dotenv(override=True)
print("🔍 PROJECT:", os.getenv("GOOGLE_CLOUD_PROJECT"))
print("🔍 CREDENTIAL FILE:", os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))
# ✅ 추가: vertexai 초기화 (프로젝트 강제 적용)

🔍 PROJECT: knu-ema
🔍 CREDENTIAL FILE: C:/Users/SAMSUNG/AppData/Roaming/gcloud/knu-ema-af3cd6fa4532.json


In [5]:
from google.oauth2 import service_account
import vertexai

# 서비스 계정 키 파일 경로
key_path = "C:/Users/SAMSUNG/AppData/Roaming/gcloud/knu-ema-af3cd6fa4532.json"

# 인증 정보 생성
credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

# ✅ 명시적으로 초기화
vertexai.init(project="knu-ema", location="asia-northeast3", credentials = credentials)


In [57]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_vertexai.embeddings import VertexAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# 1. PDF 파일 로드
loader = PyPDFLoader("James Stewart - Calculus, Early Transcendentals, International Metric Edition-CENGAGE Learning (2016).pdf")

# 2. 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)

# 3. 문서 로드 및 분할
docs = loader.load()
split_docs = text_splitter.split_documents(docs)


In [ ]:
embedding_model = VertexAIEmbeddings(
    model_name="text-multilingual-embedding-002",
    project="knu-ema"
)

# 5. FAISS VectorStore 생성
vectorstore = FAISS.from_documents(split_docs, embedding_model)

# 6. Retriever 생성
retriever = vectorstore.as_retriever()

# (선택) 확인용 출력
print(f"총 청크 수: {len(split_docs)}")
print(f"첫 청크 미리보기:\n{split_docs[0].page_content[:300]}")


Retrying vertexai.language_models._language_models._TextEmbeddingModel.get_embeddings in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: textembedding-gecko. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying vertexai.language_models._language_models._TextEmbeddingModel.get_embeddings in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: textembedding-gecko. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying vertexai.language_models._language_models._TextEmbeddingModel.get_embeddings in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: textembedding-gec